# Lab 5: Coordinate descent

We saw previously how [](S-coordinate-descent) can be used to minimize a function $f(x_1,\dots,x_p)$ of several variables. Recall that the idea is to minimize the function one variable at a time, and to repeat this process over and over:

\begin{align*}
x^{(k+1)}_1 &= \mathop{\textrm{argmin}}_x f(x, x_2^{(k)}, x_3^{(k)}, \dots, x_p^{(k)}) \\
x^{(k+1)}_2 &= \mathop{\textrm{argmin}}_x f(x_1^{(k+1)}, x, x_3^{(k)}, \dots, x_p^{(k)}) \\
x^{(k+1)}_3 &= \mathop{\textrm{argmin}}_x f(x_1^{(k+1)}, x_2^{(k+1)}, x, x_4^{(k)}, \dots, x_p^{(k)})\\
&\vdots \\
x^{(k+1)}_p &= \mathop{\textrm{argmin}}_x f(x_1^{(k+1)}, x_2^{(k+1)}, \dots , x_{p-1}^{(k+1)}, x).
\end{align*}

Under certain conditions, this process converges to a local minimum of the function $f$. To illustrate how this work with Python, let us solve the linear regression problem using coordinate descent.

## Linear regression via coordinate descent

Consider the least square problem

$$
\min_{\beta \in \mathbb{R}^p} \|y-X\beta\|_2^2, 
$$

where $y \in \mathbb{R}^n$ and $X \in \mathbb{R}^{n \times p}$ are fixed. In order to solve this problem via coordinate descent, we need to solve the one-variable problem

$$
\min_{\beta_i \in \mathbb{R}^p} \|y-X\beta\|_2^2.
$$

We will do so by computing the derivative of $\|y-X\beta\|_2^2$ with respect to $\beta_i$ and setting it to $0$. Recall from [Finding the Optimal Coefficients](sec-finding-optimal-coefficients) that 

$$
\nabla_\beta \|y-X\beta\|_2^2 = -2(X^Ty-X^TX \beta) = -2X^T(y-X\beta). 
$$

Here, $\nabla_\beta$ denotes the gradient vector obtained by differentiating with respect to $\beta_1, \dots, \beta_p$. In particular, for $1 \leq i \leq p$, the $i$-th entry of the gradient is:

$$
\frac{\partial}{\partial \beta_i} \|y-X\beta\|_2^2 = -2 X_i^T (y-X\beta), 
$$

where $X_i$ denotes the $i$-th column of $X$. Now, let $X_{-i}$ denote the matrix $X$ with the $i$-th column deleted, and let $\beta_{-i}$ be the vector $\beta$ with $i$-th entry deleted. Observe that $X\beta$ = $X_{-i} \beta_{-i} + \beta_i X_i$. It follows that 

\begin{align*}
\frac{\partial}{\partial \beta_i} \|y-X\beta\|_2^2 &= -2 X_i^T (y-X_{-i}\beta_{-i} - \beta_i X_i) \\
&= -2(X_i^T y - X_i^T X_{-i} \beta_{-i} - \beta_iX_i^T X_i).
\end{align*}

Setting the gradient to $0$ and solving for $\beta_i$, we obtain: 

$$
\beta_i^* = \mathop{\rm argmin}_{\beta_i \in \mathbb{R}} \|y-X\beta\|_2^2 = \frac{X_i^T(y-X_{-i}\beta{-i})}{X_i^T X_i}.
$$

We can therefore solve the linear regression via coordinate descent as follows. 

```{admonition} Linear regression - coordinate descent

**Input**: $y \in \mathbb{R}^n$, $X \in \mathbb{R}^{n \times p}$, initial guess $\beta^{(0)}$, tolerence $\epsilon > 0$.

Set $k = 0$. 

while $\|\left(\nabla \|y-X\beta^{(k)}\|_2^2\right)\|_2 > \epsilon$:

1. $\beta^{(k+1)} = \beta^{(k)}$.
2. For $i=1,\dots,p$: 

$$
\beta_i^{(k+1)} = \frac{X_i^T(y-X_{-i}\beta{-i}^{(k+1)})}{X_i^T X_i}. 
$$

3. Set $k = k+1$.

**Output**: A vector $\beta = \beta^{(k)} \in \mathbb{R}^p$ such that $\|\nabla \left(\|y-X\beta\|_2^2\right)\|_2 \leq \epsilon$. 

```

Let us implement this algorithm in Python. 

In [1]:
import numpy as np

# Define the norm of the gradient function

def gradnorm(X,y,beta):
    g = -2*X.T.dot(y-X.dot(beta))
    return(np.linalg.norm(g))


def least_squares(X, y, beta0 = None, epsilon=5e-3, maxit=1000):
	[n,p] = X.shape
	
	if beta0 == None:  # No initial guess was provided. Generate one at random.
		beta = np.random.rand(p)
	else:
		beta = beta0
		
	k = 0
	grad = gradnorm(X,y,beta)
	print("Iter \t Grad \t Err")
	while k < maxit and grad > epsilon:
		beta_old = np.copy(beta)
		for i in range(p):
			ind = np.arange(p)
			ind = np.delete(ind,i) # ind now contains all indices in {1,...,p} except i.
			Xi = X[:,i]
			Xmi = X[:,ind]  # All columns of X except the i-th.
			betami = beta[ind]
			beta[i] = Xi.dot(y-Xmi.dot(betami))/(Xi.dot(Xi))
				
		grad = gradnorm(X, y, beta)
		err = np.linalg.norm(y-X.dot(beta)) # Evaluates the norm of y-X*beta
		k = k + 1
		print("%d \t %f \t %f" % (k, grad, err))
	return(beta)

Let us now test our code using random data and compare its solution with the solution of the problem obtained by Scikit-learn.

In [2]:
from sklearn.linear_model import LinearRegression	
def test():
	X = np.random.rand(10,3)
	y = np.random.rand(10)
	mod = LinearRegression(fit_intercept = False)
	mod.fit(X,y)
	beta1 = mod.coef_
	beta2 = least_squares(X,y, epsilon=1e-6, maxit=100)
	print(beta1-beta2)

In [3]:
test()

Iter 	 Grad 	 Err
1 	 1.392123 	 0.996686
2 	 0.923619 	 0.905546
3 	 0.615443 	 0.862639
4 	 0.412447 	 0.843040
5 	 0.278488 	 0.834176
6 	 0.189874 	 0.830153
7 	 0.131067 	 0.828304
8 	 0.091876 	 0.827437
9 	 0.065608 	 0.827020
10 	 0.047867 	 0.826811
11 	 0.035759 	 0.826703
12 	 0.027383 	 0.826643
13 	 0.021486 	 0.826609
14 	 0.017246 	 0.826588
15 	 0.014123 	 0.826574
16 	 0.011763 	 0.826565
17 	 0.009935 	 0.826559
18 	 0.008483 	 0.826555
19 	 0.007306 	 0.826552
20 	 0.006333 	 0.826549
21 	 0.005518 	 0.826547
22 	 0.004826 	 0.826546
23 	 0.004232 	 0.826545
24 	 0.003719 	 0.826544
25 	 0.003274 	 0.826543
26 	 0.002885 	 0.826543
27 	 0.002545 	 0.826543
28 	 0.002246 	 0.826542
29 	 0.001983 	 0.826542
30 	 0.001752 	 0.826542
31 	 0.001548 	 0.826542
32 	 0.001368 	 0.826542
33 	 0.001209 	 0.826542
34 	 0.001069 	 0.826541
35 	 0.000945 	 0.826541
36 	 0.000835 	 0.826541
37 	 0.000738 	 0.826541
38 	 0.000653 	 0.826541
39 	 0.000577 	 0.826541
40 	 0.000510 	 

We obtain the same regression coefficients (up to numerical precision).

## Solving the LASSO problem using coordinate descent

Let us now consider the LASSO problem

$$
\widehat{\beta}_\textrm{LASSO} = \mathop{\textrm{argmin}}_{\beta \in \mathbb{R}^p} \|y - X\beta\|_2^2 + \alpha \|\beta\|_1.
$$

Recall that the updated derived in [](S-LASSO-soln) is 

$$
\beta_i \rightarrow \eta^S_{\lambda/\|X_i\|_2^2} \left(\frac{2 X_i^T (y-X_{-i} \beta_{-i}) }{X_i^T X_i}\right).
$$

This is almost the same update as in the linear regression problem. 

```{admonition} Exercise

Modify the above code to solve the LASSO problem instead of linear regression. Compare your solution with Scikit-learn and make sure the two versions match. Note that the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html" target="_blank">Lasso object of Scikit-learn</a> solves a slightly different (but equivalent) LASSO problem

$$
\min_{w \in \mathbb{R}^p} \frac{1}{2n}  ||y - Xw||^2_2 + \alpha  ||w||_1.
$$

Observe that setting $\alpha = \frac{\lambda}{2n}$ solves 

$$
\min_{w \in \mathbb{R}^p} \frac{1}{2n} ||y - Xw||^2_2 + \frac{\lambda}{2n}  ||w||_1 = \frac{1}{2n} \min_{w \in \mathbb{R}^p} ||y - Xw||^2_2 + \lambda  ||w||_1, 
$$

which is equivalent to our formulation.

```